In [2]:
import joblib
import pymysql
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
import platform
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from sklearn.metrics import classification_report
from pykrx import stock
import warnings
from keras.models import load_model
import time
from datetime import datetime, timedelta
warnings.filterwarnings(action='ignore')
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [26]:
conn = pymysql.connect(host='localhost', user='root', password='1234', db='stockai', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
curs = conn.cursor()
stock_code_sql = "SELECT DISTINCT stock_code FROM StockData"
curs.execute(stock_code_sql)

stock_code = curs.fetchall()
stock_code_list = pd.DataFrame(stock_code)

meansu=[]
profitdata=pd.DataFrame()
holding=pd.DataFrame()
maxsu=0
minsu=100
#출력하고자 하는 주식 개수 입력, 최대 100개
stock_n = 96
datelist=[]

current_time = datetime.now()
end_date = current_time.strftime('%Y-%m-%d')
end_date = datetime.strptime(end_date, "%Y-%m-%d")
# start_date = end_date
# start_date -= timedelta(days=300)
    
# start_date = str(start_date.date())
start_date = "2021-12-17"
end_date = str(end_date.date())
print(start_date,end_date)
try:
    # for문에 
    for i in range(0,stock_n):
        from pykrx import stock
        curs = conn.cursor()
        # sql = "SELECT * FROM StockData WHERE (data_time BETWEEN '2020-01-01' AND '2022-12-28') AND (stock_code ='{}')".format(str(stock_code_list['stock_code'][i]))
        curs.execute("SELECT * FROM StockData WHERE (data_time BETWEEN %s AND %s) AND (stock_code = %s)",(start_date, end_date, stock_code_list['stock_code'][i]))
        # 코드 앞에서 A 지우고 주식 이름 불러오기
        name = stock.get_market_ticker_name(str(stock_code_list['stock_code'][i][1:]))
        # curs.execute(sql)
        data = curs.fetchall()
        df = pd.DataFrame(data)
        
        curs.execute("SELECT * FROM stockcode WHERE stock_name=%s",name)
        data = curs.fetchall()
        score = pd.DataFrame(data)

        # df = df[-131:]
        
        # 0.00등 Nan 값으로 인식되지 않는 결측치를 Nan으로 바꿈
        df["start_open"] = df["start_open"].apply(lambda x: np.nan if x<0.00001 else x)
        df["high"] = df["high"].apply(lambda x: np.nan if x<0.00001 else x)
        df["low"] = df["low"].apply(lambda x: np.nan if x<0.00001 else x)
        df["trading_volume"] = df["trading_volume"].apply(lambda x: np.nan if x<0.00001 else x)
        df["institutional_total"] = df["institutional_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["other_corporations"] = df["other_corporations"].apply(lambda x: np.nan if x<0.00001 else x)
        df["individual"] = df["individual"].apply(lambda x: np.nan if x<0.00001 else x)
        df["foreigner_total"] = df["foreigner_total"].apply(lambda x: np.nan if x<0.00001 else x)
        df["transaction_amount"] = df["transaction_amount"].apply(lambda x: np.nan if x<0.00001 else x)
        df["short_selling"] = df["short_selling"].apply(lambda x: np.nan if x<0.01 else x)
        df["short_buying"] = df["short_buying"].apply(lambda x: np.nan if x<0.01 else x)
          
        df = df.fillna(method='bfill')
        df = df.fillna(method='ffill')
    
        df['1yd_end_close'] = df['end_close'].shift(1)
        df['1yd_trading'] = df['trading_volume'].shift(1)
        df['5일 평균 종가'] = df['end_close'].rolling(5).mean()
        df['10일 평균 종가'] = df['end_close'].rolling(10).mean()
        df['20일 평균 종가'] = df['end_close'].rolling(20).mean()
        df['60일 평균 종가'] = df['end_close'].rolling(60).mean()
        df['120일 평균 종가'] = df['end_close'].rolling(120).mean()
    
        df['5일 평균 거래량'] = df['trading_volume'].rolling(5).mean()
        df['10일 평균 거래량'] = df['trading_volume'].rolling(10).mean()
        df['20일 평균 거래량'] = df['trading_volume'].rolling(20).mean()
        df['60일 평균 거래량'] = df['trading_volume'].rolling(60).mean()
        df['120일 평균 거래량'] = df['trading_volume'].rolling(120).mean()
    
        pd.set_option('mode.chained_assignment',  None)

        df.dropna(inplace=True)
        
        # sql2 = "SELECT * FROM MacroeconomicIndicators WHERE (date_time BETWEEN '2020-01-01' AND '2022-12-28')"
        curs.execute("SELECT * FROM MacroeconomicIndicators WHERE (date_time BETWEEN %s AND %s)",(start_date, end_date))
        # curs.execute(sql2)
        data2 = curs.fetchall()

        df2 = pd.DataFrame(data2)
        
        # df2 = df2[-131:]

        df2['코스피 5일 평균 종가'] = df2['kospi'].rolling(5).mean()
        df2['코스피 20일 평균 종가'] = df2['kospi'].rolling(20).mean()
        df2['코스피 60일 평균 종가'] = df2['kospi'].rolling(60).mean()
        df2['코스피 120일 평균 종가'] = df2['kospi'].rolling(120).mean()
    
        df2.dropna(inplace=True)
        
        df_m = pd.merge(df, df2, left_on='data_time',right_on='date_time',how='inner')
        
        #merge하면서 생기는 id 행들 제거
        df_m.drop(columns=['id_x','id_y'],inplace=True)
                                            
        # 컬럼 제작
        df_m['전일종가대비 당일 시가비율'] = df_m['1yd_end_close']/df_m['start_open']
        df_m['당일종가 대비 당일 고가 비율'] = df_m['high']/df_m['end_close']
        df_m['당일종가 대비 당일 저가 비율'] = df_m['low']/df_m['end_close']
        df_m['당일 종가 대비 전일 종가 비율'] = df_m['end_close']/df_m['1yd_end_close']
        df_m['전일 거래량 대비 당일 거래량 비율'] = df_m['trading_volume']/df_m['1yd_trading']
        df_m['5일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['5일 평균 종가']
        df_m['10일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['10일 평균 종가']
        df_m['20일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['20일 평균 종가']
        df_m['60일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['60일 평균 종가']
        df_m['120일 평균 종가 대비 당일 종가 비율'] = df_m['end_close']/df_m['120일 평균 종가']

        df_m['5일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['5일 평균 거래량']
        df_m['10일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['10일 평균 거래량']
        df_m['20일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['20일 평균 거래량']
        df_m['60일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['60일 평균 거래량']
        df_m['120일평균거래량대비당일거래량'] = df_m['trading_volume']/df_m['120일 평균 거래량']

        df_m['코스피지수5일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 5일 평균 종가']
        df_m['코스피지수20일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 20일 평균 종가']
        df_m['코스피지수60일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 60일 평균 종가']
        df_m['코스피지수120일평균종가대비당일종가비율'] = df_m['kospi']/df_m['코스피 120일 평균 종가']
        
        f_col = ['start_open', 'high', 'low', 'end_close',
               'trading_volume', 'transaction_amount', 'end_rate_change',
               'institutional_total', 'other_corporations', 'individual',
               'foreigner_total', 'short_selling', 'short_buying', 'short_importance',
               '1yd_end_close', '1yd_trading', '5일 평균 종가', '10일 평균 종가', '20일 평균 종가',
               '60일 평균 종가', '120일 평균 종가', '5일 평균 거래량', '10일 평균 거래량', '20일 평균 거래량',
               '60일 평균 거래량', '120일 평균 거래량', 'kospi', 
                 'inflation', 'treasury_bonds', 'tightening','normality', 
                 'powell', 'dispute', 'japan', 'volume', 'chairman',
                 'remarks', 'thought', 'effect', 'anxiety', 'buying', 
                 'volatility', 'early_stage', 'decline', 'learning_result',
               'america_top_500', 'gold', 'copper', 'k_gov3', 'usd_k', '코스피 5일 평균 종가',
               '코스피 20일 평균 종가', '코스피 60일 평균 종가', '코스피 120일 평균 종가', '전일종가대비 당일 시가비율',
               '당일종가 대비 당일 고가 비율', '당일종가 대비 당일 저가 비율', '당일 종가 대비 전일 종가 비율',
               '전일 거래량 대비 당일 거래량 비율', '5일 평균 종가 대비 당일 종가 비율', '10일 평균 종가 대비 당일 종가 비율',
               '20일 평균 종가 대비 당일 종가 비율', '60일 평균 종가 대비 당일 종가 비율',
               '120일 평균 종가 대비 당일 종가 비율', '5일평균거래량대비당일거래량', '10일평균거래량대비당일거래량',
               '20일평균거래량대비당일거래량', '60일평균거래량대비당일거래량', '120일평균거래량대비당일거래량',
               '코스피지수5일평균종가대비당일종가비율', '코스피지수20일평균종가대비당일종가비율', 
                 '코스피지수60일평균종가대비당일종가비율','코스피지수120일평균종가대비당일종가비율']
        
        #XGB 예측
        test_x = df_m[f_col][10:]
        model = joblib.load('model/'+name+'_model.pkl')
        XGB_pred = model.predict(test_x)
        
        
        #LSTM 전처리
        test_x = df_m[f_col]
        scaler = MinMaxScaler()
        X_s = scaler.fit_transform(test_x)
        X_s = pd.DataFrame(X_s,columns=test_x.columns)
        TEST_SIZE = 180
        WINDOW_SIZE = 10
        def make_dataset(data, window_size=10):
            feature_list = []
            for i in range(len(data) - window_size):
                feature_list.append(np.array(data.iloc[i:i+window_size]))
            return np.array(feature_list)
        test_feature= make_dataset(X_s, 10)
        
        #LSTM 예측
        model = load_model('LSTM_short/'+name+'_model.h5')
        LSTM_pred = model.predict(test_feature)
        
        res=pd.DataFrame()
        res["date"]=df_m['data_time'][10:]
        res["XGB"]=XGB_pred
        res["LSTM"]=LSTM_pred
        res["end_close"]=df_m["end_close"][10:]
        
        #모의투자 방식
        money=100000000
        stock=0
        size=int(5000000/list(res["end_close"])[0])
        # print("사이즈",size)
        score["xgb_short_pred"]
        profit=[]
        hold=[]
        '''
        XGB가 1일시 2배수 구매
        XGB가 0일시 0배수 구매
        XGB*2 = XGB개도
        int((LSTM-0.5)*10) =LSTM 구매개도
        '''
        targets=""
        tgsc=0
        if score["xgb_short_pred"][0] <= score["ltms_short_pred"][0]:
            targets="LSTM"
            tgsc=score["ltms_short_pred"][0]
        else:
            targets="XGB"
            tgsc=score["xgb_short_pred"][0]
        
        # print(res[targets])
        # if tgsc<0.5:
        #     size=0
        for end,pred,day in zip(res["end_close"],res[targets],res["date"]):
            if pred > 0.9:
                if money>end*size:
                    money-=end*size
                    stock+=1*size
            if pred > 0.6:
                if money>end*size:
                    money-=end*size
                    stock+=1*size
            if pred <= 0.1:
                if stock>=size:
                    money+=end*size*0.998
                    stock-=1*size
            if pred <= 0.4:
                if stock>=size:
                    money+=end*size*0.998
                    stock-=1*size
            profit.append((money+stock*end)/100000000)
            hold.append(stock*end)
        print(day,"현재주가",end,name,targets,"현금",money,"보유주식수",stock,"총자산가치",money+stock*end,"수익률",(money+stock*end)/100000000)
        profitdata['A'+str(stock_code_list['stock_code'][i][1:])]=profit
        holding['A'+str(stock_code_list['stock_code'][i][1:])]=hold
        # print(str(stock_code_list['stock_code'][i][1:]))
        meansu.append((money+stock*end)/100000000)
            
        #print(df_m['data_time'][10:])
        #print(name + "의 " + str(df_m['data_time'][0])+" 예측 결과 = ",pred)
        
finally:
    conn.close()
print("종합수익율",sum(meansu)/96)
profitdata["날짜"]=list(df_m['data_time'])[10:]
profitdata["종합누적수익률"]=profitdata.drop(["날짜"],axis=1).sum(axis=1)/96
holding["날짜"]=list(df_m['data_time'])[10:]

2021-12-17 2022-12-29
2022-12-23 현재주가 48650.0 메리츠화재 XGB 현금 99847796.80000001 보유주식수 0 총자산가치 99847796.80000001 수익률 0.9984779680000001
2022-12-23 현재주가 59500.0 유한양행 XGB 현금 102397499.79999997 보유주식수 0 총자산가치 102397499.79999997 수익률 1.0239749979999997
2022-12-23 현재주가 62700.0 기아 XGB 현금 77242956.80000006 보유주식수 320 총자산가치 97306956.80000006 수익률 0.9730695680000006
2022-12-23 현재주가 77800.0 SK하이닉스 LSTM 현금 1498721.8000000138 보유주식수 1083 총자산가치 85756121.80000001 수익률 0.8575612180000001
2022-12-23 현재주가 34900.0 현대건설 XGB 현금 48065306.0 보유주식수 1666 총자산가치 106208706.0 수익률 1.06208706
2022-12-23 현재주가 216500.0 삼성화재 XGB 현금 110713264.0 보유주식수 0 총자산가치 110713264.0 수익률 1.10713264
2022-12-23 현재주가 24400.0 대한항공 XGB 현금 113103266.39999996 보유주식수 0 총자산가치 113103266.39999996 수익률 1.1310326639999997
2022-12-23 현재주가 82100.0 LG LSTM 현금 107940041.20000003 보유주식수 63 총자산가치 113112341.20000003 수익률 1.1311234120000004
2022-12-23 현재주가 178500.0 포스코케미칼 LSTM 현금 2668816.0 보유주식수 960 총자산가치 174028816.0 수익률 1.74028816
2022-12-23 현재주가 32350.0 현대제철 XGB 현금 

In [14]:
holding

,A000060,A000100,A000270,A000660,A000720,A000810,A003490,A003550,A003670,A004020,...,A282330,A293490,A302440,A316140,A323410,A326030,A329180,A352820,A361610,날짜
0,0.0,9910200.0,9900000.0,0.0,9996900.0,0.0,0.0,9858000.0,9950000.0,9987600.0,...,9936000.0,9984600.0,9919000.0,0.0,0.0,0.0,9890000.0,9996000.0,0.0,2022-10-27
1,0.0,19853600.0,19860000.0,0.0,19796400.0,0.0,0.0,19592000.0,20000000.0,18860000.0,...,19872000.0,20020800.0,19552000.0,0.0,0.0,9865800.0,20148000.0,19992000.0,5086200.0,2022-10-28
2,0.0,29780400.0,29835000.0,0.0,29525400.0,0.0,0.0,29462400.0,29925000.0,27650400.0,...,30213000.0,30611700.0,29484000.0,0.0,0.0,0.0,30084000.0,30492000.0,10192000.0,2022-10-31
3,0.0,40105600.0,39780000.0,0.0,42018000.0,0.0,0.0,40374400.0,40800000.0,37982400.0,...,40176000.0,40867200.0,39676000.0,10081500.0,0.0,10179000.0,40480000.0,40992000.0,16611000.0,2022-11-01
4,0.0,50049000.0,49650000.0,0.0,52240500.0,0.0,0.0,50282000.0,49500000.0,28929600.0,...,49545000.0,50955000.0,54340000.0,19905600.0,0.0,0.0,51520000.0,32004000.0,23520000.0,2022-11-02
5,0.0,58963200.0,58050000.0,0.0,65057400.0,0.0,0.0,60115200.0,63600000.0,38244800.0,...,59454000.0,57895200.0,41964000.0,9781200.0,0.0,10335600.0,61272000.0,41832000.0,27293000.0,2022-11-03
6,0.0,66698800.0,68565000.0,0.0,75406800.0,0.0,0.0,69960800.0,73325000.0,28290000.0,...,50760000.0,50245500.0,31668000.0,0.0,0.0,20706000.0,75026000.0,30870000.0,40062400.0,2022-11-04
7,0.0,76227200.0,80520000.0,0.0,89112000.0,0.0,0.0,80947200.0,85800000.0,40212800.0,...,60264000.0,60913800.0,22048000.0,9909900.0,0.0,10822800.0,83904000.0,20748000.0,51597000.0,2022-11-07
8,0.0,85905000.0,91125000.0,4823500.0,101012400.0,0.0,0.0,91288800.0,95175000.0,50430000.0,...,68607000.0,74136300.0,11063000.0,20077200.0,0.0,21715200.0,95220000.0,10878000.0,57918000.0,2022-11-08
9,0.0,95118000.0,101100000.0,9812000.0,103423500.0,0.0,0.0,97067200.0,102837500.0,61696800.0,...,77112000.0,85656000.0,0.0,30373200.0,0.0,10996800.0,98762000.0,0.0,71383200.0,2022-11-09


In [97]:
profitdata.to_csv("창준님의비밀롱기누스.csv",index=False)

In [ ]:
8.3%수익률
8.6%수익률